<a href="https://colab.research.google.com/github/ArkS0001/AI-Agent-Atom-of-Thought-2026/blob/main/granite3_3_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch torchvision torchaudio accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import torch

model_path="ibm-granite/granite-3.3-2b-instruct"
device="cuda"
model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map=device,
        torch_dtype=torch.bfloat16,
    )
tokenizer = AutoTokenizer.from_pretrained(
        model_path
)

conv = [{"role": "user", "content":"Take a current environmental issue and work backward to devise an innovative prevention strategy that could have averted the problem."}]

input_ids = tokenizer.apply_chat_template(conv, return_tensors="pt", thinking=True, return_dict=True, add_generation_prompt=True).to(device)

set_seed(42)
output = model.generate(
    **input_ids,
    max_new_tokens=8192,
)

prediction = tokenizer.decode(output[0, input_ids["input_ids"].shape[1]:], skip_special_tokens=True)
print(prediction)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

<think>To address this task, I'll choose the environmental issue of plastic pollution in our oceans as a current problem. I'll then work backward to devise an innovative prevention strategy that could have mitigated this issue.

### Current Environmental Issue: Plastic Pollution in Oceans

**Problem Description:**
- Millions of tons of plastic waste enter oceans annually, causing severe harm to marine life, ecosystems, and human health.
- Plastic debris persists in the environment for centuries, breaking down into microplastics that contaminate food chains.

### Working Backward to Develop a Prevention Strategy

1. **Identify Root Causes:**
   - **Consumer Behavior:** Over-reliance on single-use plastics (e.g., bags, straws, bottles).
   - **Industrial Practices:** Lack of sustainable packaging solutions and inadequate waste management systems.
   - **Government Policies:** Insufficient regulations and enforcement regarding plastic production and disposal.

2. **Develop a Prevention St

## AGENT

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, re, json, random

# ──────────────────────────────────────────
# 1. Load Granite-3.3-2B-Instruct
# ──────────────────────────────────────────
model_name = "ibm-granite/granite-3.3-2b-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.eval()

# ──────────────────────────────────────────
# 2. Define Available Tools
# ──────────────────────────────────────────
TOOLS = [
    {
        "name": "calculator",
        "description": "Evaluate a basic arithmetic expression",
        "arguments": {
            "expression": {
                "type": "string",
                "description": "Arithmetic using digits and + - * / ( )"
            }
        }
    },
    {
        "name": "get_current_weather",
        "description": "Fetch current weather for a location",
        "arguments": {
            "location": {
                "type": "string",
                "description": "City name, e.g. 'Paris, FR'"
            }
        }
    }
]

def calculator_tool(expr: str) -> str:
    if not re.fullmatch(r"[0-9+\-*/ ().]+", expr):
        return "Invalid expression"
    try:
        return str(eval(expr))
    except:
        return "Error"

def get_current_weather(loc: str) -> str:
    temp = random.randint(-5, 35)
    return f"The current temperature in {loc} is {temp}°C."

# ──────────────────────────────────────────
# 3. Run One Tool-Calling Turn
# ──────────────────────────────────────────
def run_with_tools(question: str) -> str:
    conv = [{"role": "user", "content": question}]

    # Initial pass: model may emit a tool_call
    inputs = tokenizer.apply_chat_template(
        conv,
        tools=TOOLS,
        thinking=True,
        return_tensors="pt",
        return_dict=True,
        add_generation_prompt=True
    ).to(device)

    out = model.generate(**inputs, max_new_tokens=512)
    chunk = tokenizer.decode(
        out[0, inputs["input_ids"].shape[1]:],
        skip_special_tokens=False
    )

    # Look for the <|tool_call|> JSON payload
    match = re.search(r"<\|tool_call\|>(\{.*?\})", chunk, re.DOTALL)
    if not match:
        return chunk.strip()  # No tool call

    # Parse the JSON and execute the tool
    call = json.loads(match.group(1))
    name = call["name"]
    args = call["arguments"]

    if name == "calculator":
        result = calculator_tool(args["expression"])
    elif name == "get_current_weather":
        result = get_current_weather(args["location"])
    else:
        result = f"Error: unknown tool {name}"

    # Append the model’s reasoning and tool response
    reasoning = chunk.split("<|tool_call|>")[0]
    conv.append({"role": "assistant", "content": reasoning})
    conv.append({
        "role": "tool_response",
        "name": name,
        "content": json.dumps({"result": result})
    })

    # Final pass: ask Granite to finish its answer
    final_inputs = tokenizer.apply_chat_template(
        conv,
        tools=TOOLS,
        thinking=True,
        return_tensors="pt",
        return_dict=True,
        add_generation_prompt=True
    ).to(device)

    final_out = model.generate(**final_inputs, max_new_tokens=256)
    final_text = tokenizer.decode(
        final_out[0, final_inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    )
    return final_text.strip()

# ──────────────────────────────────────────
# 4. Example
# ──────────────────────────────────────────
if __name__ == "__main__":
    q = (
        "What is 47 * 13 minus 5? "
        "Also, tell me the current weather in Paris."
    )
    print("Answer:\n", run_with_tools(q))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Answer:
 <|tool_call|>[{"name": "calculator", "arguments": {"expression": "47 * 13 minus 5"}}, {"name": "get_current_weather", "arguments": {"location": "Paris"}}]<|end_of_text|>


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, re, json, random

# ── 1. Load Granite-3.3-2B-Instruct ─────────────────────────────
model_name = "ibm-granite/granite-3.3-2b-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", torch_dtype=torch.bfloat16
)
model.eval()

# ── 2. Define Tools ────────────────────────────────────────────
TOOLS = [
    {
        "name": "calculator",
        "description": "Evaluate a basic arithmetic expression",
        "arguments": {
            "expression": {"type": "string", "description": "Digits and + - * / ( ) only"}
        }
    },
    {
        "name": "get_current_weather",
        "description": "Fetch current weather for a location",
        "arguments": {
            "location": {"type": "string", "description": "City, e.g. 'Paris, FR'"}
        }
    }
]

def calculator_tool(expr: str) -> str:
    if not re.fullmatch(r"[0-9+\-*/ ().]+", expr):
        return "Invalid expression"
    try:
        return str(eval(expr))
    except:
        return "Error"

def get_current_weather(loc: str) -> str:
    temp = random.randint(-5, 35)
    return f"The current temperature in {loc} is {temp}°C."

# ── 3. Single Turn with Multiple Tools ──────────────────────────
def run_with_multiple_tools(question: str) -> str:
    conv = [{"role": "user", "content": question}]

    # Initial generation: model may emit one <|tool_call|> with a JSON array
    inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(device)

    out = model.generate(**inputs, max_new_tokens=512)
    chunk = tokenizer.decode(
        out[0, inputs["input_ids"].shape[1]:],
        skip_special_tokens=False
    )

    # Extract array of tool calls
    match = re.search(r"<\|tool_call\|>(\[\{.*?\}\])", chunk, re.DOTALL)
    if not match:
        return chunk.strip()

    calls = json.loads(match.group(1))  # List of {name, arguments}

    # Split out the model's internal reasoning before the call marker
    reasoning = chunk.split("<|tool_call|>")[0]
    conv.append({"role": "assistant", "content": reasoning})

    # Execute each tool and append its response
    for call in calls:
        name, args = call["name"], call["arguments"]
        if name == "calculator":
            result = calculator_tool(args["expression"])
        elif name == "get_current_weather":
            result = get_current_weather(args["location"])
        else:
            result = f"Error: unknown tool {name}"
        conv.append({
            "role": "tool_response",
            "name": name,
            "content": json.dumps({"result": result})
        })

    # Final generation: model sees all tool responses and completes the answer
    final_inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(device)

    final_out = model.generate(**final_inputs, max_new_tokens=256)
    final_text = tokenizer.decode(
        final_out[0, final_inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    )
    return final_text.strip()

# ── 4. Example Usage ───────────────────────────────────────────
if __name__ == "__main__":
    q = "Compute 47 * 13 minus 5 and also tell me the weather in Paris."
    print("Answer:\n", run_with_multiple_tools(q))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Answer:
 The result of the arithmetic expression 47 * 13 minus 5 is 606. The current temperature in Paris is 35°C.
